In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

log = pd.read_csv('C:/Users/12345/Documents/SkillFactory/module_1_11/log.xls',  sep = ',',header = None)
log.columns = ['user_id', 'time', 'bet', 'win']
us = pd.read_csv('C:/Users/12345/Documents/SkillFactory/module_1_11/users.xls', encoding='KOI8-R',  sep = '\t',header = None)#utf-8 cp1252 ISO-8859-1
us.columns = ['user_id', 'email', 'geo']

log['bet'].fillna(0,inplace=True)
#log['bet'].value_counts()

def fillna_win(row):
    if not pd.isna(row.win):
        return row.win
    elif pd.isna(row.win) and row.bet == 0:
        return 0
    else:
        return -row.bet
  
# Применяем функцию  
new_win = log.apply(lambda row: fillna_win(row), axis=1)  
  
# Заменяем старый столбец с пропусками на новый без пропусков  
log['win'] = new_win
len(log['win'][log['win']<0])


def fillna_net(row):
    if row.win < 0:
        return row.win
    else:
        return row.win-row.bet
  
# Применяем функцию  
new_net = log.apply(lambda row: fillna_net(row), axis=1)  
  
# получаем net
log['net'] = new_net
(log['net'][log['net']>0].median())

# количество минимальной ставки
min_bet = log['bet'][log['bet']>0].min()
log['bet'].value_counts()[min_bet]

us.user_id = us.user_id.apply(lambda x: x.lower())  
# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])  

df = pd.merge(log, us, on='user_id') 
len(df)
df

,user_id,time,bet,win,net,email,geo
0,user_919,[2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,[2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,[2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,[2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,[2019-03-02 4:23:36,300.0,-300.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,[2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,[2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,[2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,[2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


In [7]:
df.groupby('user_id').net.sum().median()

1986.0

In [12]:
# Сама функция и решение
def time_delta_0(group):
    first_time_in = group.time.min()
    first_bet = group[group.bet != 0].time.min()
    return (first_bet - first_time_in).days

days_to_bet = df.groupby('user_id').apply(time_delta_0)
days_to_bet.mean()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [243]:
def mean_time_bet(users_group):
    return users_group[users_group.bet != 0].time.min() - users_group.time.min()
df.groupby('user_id').apply(mean_time_bet).mean()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [240]:
df.groupby('user_id').net.median().median()

0.0

In [241]:
df.groupby('user_id').win.median().median()  

0.0

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

log = pd.read_csv('C:/Users/12345/Documents/SkillFactory/module_1_11/log.xls',  sep = ',',header = None)
log.columns = ['user_id', 'time', 'bet', 'win']
us = pd.read_csv('C:/Users/12345/Documents/SkillFactory/module_1_11/users.xls', encoding='KOI8-R',  sep = '\t',header = None)#utf-8 cp1252 ISO-8859-1
us.columns = ['user_id', 'email', 'geo']

# Приведем признак user_id к одному формату в обоих датасетах  
us.user_id = us.user_id.apply(lambda x: x.lower())
log.time = log.time.apply(lambda x: str(x).replace('[', ''))
# Приводим время к нужному формату
log.time = pd.to_datetime(log.time)
# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']
def fillna_win(row):
    # 1. Если значение в столбце win существует (не пропуск) - вернуть это же значение. Это значит, что человек выиграл.
    if pd.notna(row['win']):
        return row
    # Если в столбце bet нет пропуска, а в столбце win есть пропуск, вернуть отрицательное значение столбца bet (проигрыш).
    elif pd.notna(row['bet']):
        row['win'] = row['bet'] * (-1)
        return row
    # 2. Если вместо значения в столбце win и в столбце bet пропуски, вернуть 0.
    else:
        row['win'] = 0
        row['bet'] = 0
        return row
def calculate_net(row):
    if row.win < 0:
        return row.win
    else:
        return row.win - row.bet
# Применяем функцию  
new_win = log.apply(lambda row: fillna_win(row), axis=1)  
# Заменяем старый столбец с пропусками на новый без пропусков  
log = new_win
log['net'] = log.apply(lambda row: calculate_net(row), axis=1)
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
# Сливаем вместе два датасета
df = pd.merge(log, us, on='user_id')
#df.groupby('user_id').net.mean().head()
df.geo = df.geo.str.lower()
df.groupby('geo', sort = True).win.sum().sort_values(ascending = False)
sample2 = df.groupby('geo').time.count()
sample2

geo
арзангелтск         96
воронеж             88
екатеринбург        49
ижевск              61
казань              66
краснодар           86
красноярск          56
москва              61
пермь               55
санкт-петербург    115
ставрополь          36
тюмень              32
хабаровск           60
ярославль           89
Name: time, dtype: int64

In [52]:
# Сама функция и решение
def time_delta_0(group):
    first_time_in = group.time.min()
    first_bet = group[group.bet != 0].time.min()
    return (first_bet - first_time_in).days

days_to_bet = df[df.bet>0].groupby('geo').bet.mean()
days_to_bet.max()//days_to_bet.min()


127.0

In [14]:
# Сама функция и решение
def time_delta_0(group):
    first_time_in = group.time.min()
    first_bet = group[group.bet != 0].time.min()
    return (first_bet - first_time_in).days

days_to_bet = df.groupby('user_id').apply(time_delta_0)
days_to_bet.mean()

46.6

In [15]:
def mean_time_bet(users_group):
    return users_group[users_group.bet != 0].time.min() - users_group.time.min()
df.groupby('user_id').apply(mean_time_bet).mean()

Timedelta('47 days 01:34:33.930000')

In [33]:
len(log.user_id.unique())
df[df.bet == 0].time.count()

505

In [36]:
len_goup = len(log.user_id.unique())
def mean_time_bet(users_group):
    day_no_bet = users_group[users_group.bet == 0].time.count()
    day_with_bet = users_group[users_group.bet != 0].time.count()
    if day_with_bet > 0:
        return day_no_bet#/ len_goup
    else: return 0

#users_group[users_group.bet == 0].time.sum() - users_group.time.min()
df.groupby('user_id').apply(mean_time_bet).sum()// len_goup

5

In [236]:
len(log['bet'][log['bet']>0])/len(log['bet'])

0.47918781725888326

In [216]:
log['bet'][log['bet']>0].mean()

6785.738144329897

In [217]:
log['net'][log['bet']>0].mean()

20421.892783505155

In [218]:
#среднее значение потерь
log['net'][log['net']<0].mean()

-3372.743515850144

In [222]:
# минимальная ставка
min_bet = log['bet'][log['bet']>0].min()
log['bet'].value_counts()[min_bet]

48

In [179]:
log = pd.read_csv('C:/Users/12345/Documents/SkillFactory/module_1_11/log.xls',  sep = ',')
log.columns = ['user_id', 'time', 'bet', 'win']
log.time = log.time.str.replace('[', '')
log.user_id = log.user_id.str.strip().replace('#error', '')
log['time'] = pd.to_datetime(log['time'])
log = log.dropna(axis=0)
log['hour'] = log['time'].dt.hour

In [188]:
#log.dropna(subset=['time'], axis=0)
log

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN


In [131]:
log['time'].isna()#.sum()

0      False
1      False
2      False
3      False
4      False
       ...  
990    False
991    False
992    False
993    False
994    False
Name: time, Length: 984, dtype: bool

In [73]:
log.dropna(['time'],axis=1)


TypeError: dropna() got multiple values for argument 'axis'

In [105]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  999 non-null    object 
 1   time     984 non-null    object 
 2   bet      485 non-null    float64
 3   win      138 non-null    float64
dtypes: float64(2), object(2)
memory usage: 31.3+ KB


In [88]:
len(log)

999

In [135]:
log.time = log.time.str.replace('[', '')
log['time'] = pd.to_datetime(log['time'])

In [68]:
log.dropna(['user_id','time'], axis=1).head()

TypeError: dropna() got multiple values for argument 'axis'

In [109]:
log.drop_duplicates(subset=['time', 'user_id'], keep=False)
len(log)
log.reindex()

,user_id,time,bet,win
0,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,NaN
...,...,...,...,...
994,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
995,#error,NaN,10054.0,29265.0
996,#error,NaN,10454.0,NaN
997,#error,NaN,1000.0,NaN


In [191]:
log['bet'].fillna(0)
log['bet'].value_counts()
#log

KeyError: 'bet'

In [2]:
#log = pd.read_csv('log.csv', header = None)
#log.columns = ['user_id','time','bet','win']
# приводим столбец time к формату даты.
log.time = log.time.str.replace('[', '')
log['time'] = pd.to_datetime(log['time'])
max_time = log.time.max()
print(max_time)
# удаляем пропуски в столбце bet
log['bet'] = log['bet'].fillna(0)
# Преобразуем столбец win. заполняем пропуски в зависимости от условия 0 - ставки не было, значение со знаком "-" - проигрыш, "+ - выигрыш"
def fillna_win(row):
    if pd.isna(row.win) == False:
        return row.win
    elif pd.isna(row.win) and row.bet == 0:
        return 0
    else:
        return -row.bet
# Применяем функцию
new_win = log.apply(lambda row: fillna_win(row), axis=1)
# Заменяем старый столбец с пропусками на новый без пропусков
log['win'] = new_win
# Создаем новый столбец в котором записываем чистый выигрыш = выигрыш - ставка
def fillna_net(row):
    if row.win <= 0:
        return row.win
    else:
        return row.win-row.bet
# Применяем функцию
log['net'] = log.apply(lambda row: fillna_net(row), axis=1)
#users = pd.read_csv('users.csv', encoding = 'koi8-r', sep = '\t')
#users.columns = ['user_id', 'email', 'geo']
# Приведем признак user_id к одному формату в обоих датасетах
users.user_id = users.user_id.apply(lambda x: x.lower())
# Избавимся от ошибок в user_id
log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
df = pd.merge(log, users, on='user_id')
# группируем по городам, находим среднее и делим максимальное значение на минимальное
#df_bet = df.groupby('city').bet.mean()
#df_bet.max()/df_bet.min() (edited) 

2019-04-20 18:10:07


NameError: name 'users' is not defined

In [111]:
pd.to_datetime('1993-06-08')

Timestamp('1993-06-08 00:00:00')

In [129]:
log.time.dt.month.value_counts() 

1    291
3    264
2    259
4    170
Name: time, dtype: int64

In [143]:
#log['minute'] = log['time'].dt.dayofweek 
log['hour'] = log['time'].dt.hour 

In [137]:
log['holiday'] = log['time'].apply(lambda x: True if x.dayofweek >=5 else False)  #.dt.dayofweek

In [144]:
log['hour'].value_counts()

10.0    52
3.0     50
1.0     50
8.0     49
6.0     47
17.0    45
5.0     45
4.0     43
12.0    42
18.0    42
22.0    41
14.0    41
13.0    41
2.0     40
23.0    39
9.0     39
0.0     37
20.0    37
15.0    37
21.0    35
11.0    34
19.0    33
16.0    33
7.0     32
Name: hour, dtype: int64

In [172]:
log.dropna(subset=['time'], axis=0).head()

,user_id,time,bet,win,hour
0,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN,14.0
1,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN,16.0
2,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN,17.0
3,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN,21.0
4,Запись пользователя № - user_917,2019-01-01 23:34:55,156789.0,NaN,23.0


In [ ]:
seconds_series = pd.Series(pd.date_range("2000-01-01", periods=3, freq="m"))
seconds_series
0   2000-01-01 00:00:00
1   2000-01-01 00:00:01
2   2000-01-01 00:00:02
dtype: datetime64[ns]
seconds_series.dt.second

In [153]:
def part_of_day(time):

    if 0<time <=5:
        return 'ночь'
    elif 6<time <=11:
        return 'утро'
    elif 12<time <=17:
        return 'день'
    elif 18<time <=23:
        return 'вечер'

In [156]:
log['part_day'] = log['hour'].apply(part_of_day)

In [160]:
log['part_day'].value_counts()

ночь     228
утро     206
день     197
вечер    185
Name: part_day, dtype: int64

In [150]:
seconds_series = pd.Series(pd.date_range(log['time'], periods=4, freq="h"))

#seconds_series.dt.second

TypeError: Cannot convert input [0     2019-01-01 14:51:16
1     2019-01-01 16:31:16
2     2019-01-01 17:17:51
3     2019-01-01 21:31:18
4     2019-01-01 23:34:55
              ...        
994   2019-04-20 09:59:58
995                   NaT
996                   NaT
997                   NaT
998                   NaT
Name: time, Length: 999, dtype: datetime64[ns]] of type <class 'pandas.core.series.Series'> to Timestamp

In [151]:
log

,user_id,time,bet,win,minute,holiday,hour
0,Запись пользователя № - user_973,2019-01-01 14:51:16,NaN,NaN,1.0,False,14.0
1,Запись пользователя № - user_903,2019-01-01 16:31:16,NaN,NaN,1.0,False,16.0
2,Запись пользователя № - user_954,2019-01-01 17:17:51,NaN,NaN,1.0,False,17.0
3,Запись пользователя № - user_954,2019-01-01 21:31:18,NaN,NaN,1.0,False,21.0
4,Запись пользователя № - user_917,2019-01-01 23:34:55,156789.0,NaN,1.0,False,23.0
...,...,...,...,...,...,...,...
994,Запись пользователя № - user_984,2019-04-20 09:59:58,9754.0,NaN,5.0,True,9.0
995,#error,NaT,10054.0,29265.0,NaN,False,NaN
996,#error,NaT,10454.0,NaN,NaN,False,NaN
997,#error,NaT,1000.0,NaN,NaN,False,NaN


In [56]:
import numpy as np
apartment = np.array([59.50, 31.40, 19, 22, 60550, 2])
# вычислим долю жилой площади в квартире
share_living_space = apartment[1]/apartment[0]

In [57]:
apartment = np.delete(apartment, [0, 1])

In [58]:
apartment

array([1.900e+01, 2.200e+01, 6.055e+04, 2.000e+00])

In [59]:
apartment = np.append(apartment, share_living_space)

In [60]:
apartment

array([1.90000000e+01, 2.20000000e+01, 6.05500000e+04, 2.00000000e+00,
       5.27731092e-01])

In [61]:
apartment[4]

0.5277310924369748

In [62]:
share_living_space

0.5277310924369748

In [66]:
t[6]

31

In [65]:
t.mean()

21.25

In [67]:
zp = np.array([2,3,2.5])
zp = 72*zp
zp

array([144., 216., 180.])

In [68]:
zp+np.array([120,150,90])+np.array([130,130,130])

array([394., 496., 400.])

In [69]:
2*np.array([4,5])-3*np.array([2,1])+5*np.array([1,0])

array([7, 7])

In [70]:
400*np.array([1,5,3,6])-200*np.array([3,4,5,9])

array([-200, 1200,  200,  600])